# Tweets stream on socket as source

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import dotenv
import socket
import json
import os


class TwitterAuthenticator():
    """
    Class for authenticating twitter client api.
    """
    def authenticate_twitter_app(self):
        cred = dotenv.dotenv_values()
        auth = OAuthHandler(cred.get('TWI_CONSUMER_KEY'), cred.get('TWI_CONSUMER_SECRET'))
        auth.set_access_token(cred.get('TWI_ACCESS_TOKEN'), cred.get('TWI_ACCESS_TOKEN_SECRET'))
        return auth


class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, c_socket, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, TwitterListener(c_socket))

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list, languages=["en"])


class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, csocket):
        self.client_socket = csocket
        
    def on_data(self, raw_data):
        try:
            msg = json.loads(raw_data)
            print(msg['text'].encode('utf-8'))
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True
        
        except BaseException as e:
            print("Error on data: %s" % str(e))
            return True
        
    def on_error(self, status_code):
        print(status_code)
        return True

In [ ]:
if __name__ == '__main__':
    s = socket.socket()
    host = 'localhost'
    port = 5555
    s.bind((host, port))
    print("Listening on port: %s" % str(port))
    
    s.listen(5)
    c, addr = s.accept()
    
    print("Received request from: " + str(addr))
    hash_tag_list = ['covid']
    stream = TwitterStreamer()
    # tweet_analyzer = TweetAnalyzer()
    tweets = stream.stream_tweets(c, hash_tag_list)
    